In [1]:
import numpy as np
import time
from random import randint
from numpy import linalg as LA
from collections import Counter

In [3]:
# Read in data
with open('../pa1train.txt') as f:
    train = f.readlines()
train = [x.strip() for x in train] 

with open('../pa1validate.txt') as f:
    val = f.readlines()
val = [x.strip() for x in val] 

with open('../pa1test.txt') as f:
    test = f.readlines()
test = [x.strip() for x in test] 

In [4]:
# Read in proj matrix and remove spacing
with open('../projection.txt') as f:
    proj = f.readlines()
proj = [x.strip() for x in proj] 

for i in range (0,len(proj)):
    proj[i] = proj[i].split(" ")
    
for i in range(0,len(proj)):
    proj[i] = [float(j) for j in proj[i]]

In [5]:
# create empty lists for data sorting
trainData = []
trainLabel = []

valData = []
valLabel = []

testData = []
testLabel = []

In [6]:
# Remove spaces and split into label/data
for i in range (0,len(train)):
    train[i] = train[i].split(" ")
    trainLabel.append(train[i][784])
    trainData.append(train[i][:784])

for i in range (0,len(val)):
    val[i] = val[i].split(" ")
    valLabel.append(val[i][784])
    valData.append(val[i][:784])

for i in range (0,len(test)):
    test[i] = test[i].split(" ")
    testLabel.append(test[i][784])
    testData.append(test[i][:784])

In [8]:
# convert label string to int
trainLabel = [int(i) for i in trainLabel]

valLabel = [int(i) for i in valLabel]

testLabel = [int(i) for i in testLabel]

In [9]:
# convert data string to int
for i in range(0,len(trainData)):
    trainData[i] = [int(j) for j in trainData[i]]
    
for i in range(0,len(valData)):
    valData[i] = [int(j) for j in valData[i]]
    
for i in range(0,len(testData)):
    testData[i] = [int(j) for j in testData[i]]

In [10]:
# create empty lists
projMat = []
projRow = []
x = 0

In [11]:
# Apply Matrix Multiplication to creat projection matrix
for i in range (0,len(trainData)):
    for k in range (0,len(proj[0])):
        for j in range (0, len(proj)):
            x += (trainData[i][j] * proj[j][k])
        projRow.append(x)
        x = 0
    projMat.append(projRow)
    projRow = []

In [12]:
# create list of k values
k = [1,5,9,15]

In [13]:
def KNN(training, toPred, biggestkVal):
    allDist = []
    kResults = []

    # calculate all the distances between vector to predict and training data
    for i in range (0,len(training)):
        dist = LA.norm(np.array(training[i])-np.array(toPred))
        allDist.append((i,dist))
    
    # sort all values by distance
    allDist.sort(key=lambda tup: tup[1]) 

    # check for ties
    i = 0
    while(len(kResults) != biggestkVal):
        # if there is a tie, randomly choose one of the values
        if(allDist[i][1] == allDist[i+1][1]):
            coin = randint(i,i+1)
            kResults.append(allDist[coin][0])
        else:
            kResults.append(allDist[i][0])
        i += 1
    
    return kResults    

In [105]:
output = KNN(trainData,testData[30],15)

In [112]:
output[:10]

[111, 178, 141, 144, 22, 80, 32, 172, 113, 9]

In [107]:
data = Counter([trainLabel[x] for x in output])

In [108]:
data.most_common()[0][0]

0

In [109]:
set(trainLabel)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [17]:
dud = Counter([1,1,1,2,2,2,3,3,4,4])

In [18]:
dud.most_common()[:3]

[(1, 3), (2, 3), (3, 2)]

In [139]:
testPred = []

In [142]:
t0 = time.time()
pred = []
for i in range (0,len(testData)):
    dup = 0
    
    # outputs largest K values of training data predictions on one test data point
    output = KNN(trainData,testData[i],15)
    
    for m in range (0,len(k)):
        # get the current k value
        curK = k[m]
        
        # counts label occurences in the predictions
        data = Counter([trainLabel[x] for x in output[:curK]])
    
        # if there is only one common label append it
        if(len(data.most_common()) == 1):
            pred.append(data.most_common()[0][0])
        else:
            # check all top common data for duplicates
            for j in range (0,len(data.most_common())-1):
                # if current top count == next top count then add duplicates
                if(data.most_common()[j][1] == data.most_common()[j+1][1]):
                    dup += 1
                else:
                    break
            # pick a random duplicate
            pred.append(data.most_common()[randint(0,dup)][0])
    testPred.append(pred)
    pred = []
t1 = time.time()

In [143]:
total = t1-t0

In [144]:
allCount = []
counter = 0

In [145]:
total

216.2346441745758

In [146]:
len(trainPred)

2000

In [148]:
for j in range (0, len(k)):
    for i in range (0,len(testLabel)):
        if(testLabel[i] == testPred[i][j]):
            counter += 1
    allCount.append(counter)
    counter = 0

In [125]:
trainPred[0][0]

0

In [153]:
testError = []

In [154]:
for i in range (0,len(allCount)):
    testError.append((len(testLabel)-allCount[i])/len(testLabel))

In [155]:
allCount

[874, 868, 858, 850]

In [157]:
testError

[0.126, 0.132, 0.142, 0.15]